In [4]:

from transformers import AutoProcessor, AutoModelForCausalLM

In [5]:
path = "google/functiongemma-270m-it"

In [6]:
processor = AutoProcessor.from_pretrained("google/functiongemma-270m-it", device_map="auto")
model = AutoModelForCausalLM.from_pretrained("google/functiongemma-270m-it", dtype="auto", device_map="auto")

In [7]:
weather_function_schema = {
    "type": "function",
    "function": {
        "name": "get_current_temperature",
        "description": "Gets the current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city name, e.g. San Francisco",
                },
            },
            "required": ["location"],
        },
    }
}

In [8]:
message = [
    {
        "role" : "developer",
        "content" : "You are the model that can do function calling with the following functions."
    },
    {
        "role": "user",
        "content": "What's the current temperature in New York?",
    }
]

In [9]:
inputs = processor.apply_chat_template(message, tools= [ weather_function_schema], add_generation_prompt= True, return_dict= True , return_tensors="pt")

In [10]:
out = model.generate(**inputs.to(model.device), pad_token_id = processor.eos_token_id, max_new_tokens= 512)

In [11]:
output = processor.decode(out[0], skip_special_tokens=True)

In [12]:
print(output)

developer
You are the model that can do function calling with the following functions.<start_function_declaration>declaration:get_current_temperature{description:<escape>Gets the current temperature for a given location.<escape>,parameters:{properties:{location:{description:<escape>The city name, e.g. San Francisco<escape>,type:<escape>STRING<escape>}},required:[<escape>location<escape>],type:<escape>OBJECT<escape>}}<end_function_declaration>
user
What's the current temperature in New York?
model
<start_function_call>call:get_current_temperature{location:<escape>New York<escape>}<end_function_call>


In [13]:
output = processor.decode(out[0][len(inputs["input_ids"][0]):], skip_special_tokens=True)

print(output)

<start_function_call>call:get_current_temperature{location:<escape>New York<escape>}<end_function_call>
